<a href="https://colab.research.google.com/github/alessandroweber/AgendaContatos/blob/main/Titanic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [2]:
# Carregar o DataFrame
df = pd.read_csv('train.csv')

In [3]:
# 1. Análise Inicial
print("1. Análise Inicial\n")
print("Resumo do DataFrame:")
df.info()
print("\nResumo estatístico para colunas numéricas:")
print(df.describe())

1. Análise Inicial

Resumo do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

Resumo estatístico para colunas numéricas:
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000 

In [4]:
# 2. Tratamento de Dados Ausentes
print("\n2. Tratamento de Dados Ausentes")
# Preencher 'Age' com a média para cada Pclass
df['Age'] = df.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))
print("Valores ausentes em 'Age' preenchidos com a média por Pclass.")
# Preencher 'Embarked' com a moda
most_frequent_embarked = df['Embarked'].mode()[0]
df['Embarked'].fillna(most_frequent_embarked, inplace=True)
print("Valores ausentes em 'Embarked' preenchidos com a moda.")
# Remover a coluna 'Cabin'
df.drop('Cabin', axis=1, inplace=True)
print("Coluna 'Cabin' removida.")
# Verificação final de nulos
print("\nContagem de valores nulos após o tratamento:")
print(df.isnull().sum())


2. Tratamento de Dados Ausentes
Valores ausentes em 'Age' preenchidos com a média por Pclass.
Valores ausentes em 'Embarked' preenchidos com a moda.
Coluna 'Cabin' removida.

Contagem de valores nulos após o tratamento:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


/tmp/ipython-input-2443593869.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(most_frequent_embarked, inplace=True)


In [5]:
# 3. Tratamento de Outliers (Coluna 'Fare')
print("\n3. Tratamento de Outliers na coluna 'Fare'")
# Visualização opcional antes da remoção
plt.figure(figsize=(8, 6))
df.boxplot(column='Fare')
plt.title('Boxplot de Fare (Antes da Remoção de Outliers)')
plt.ylabel('Fare')
plt.savefig('fare_boxplot_before.png')
plt.close()


3. Tratamento de Outliers na coluna 'Fare'


In [6]:
# Cálculo dos limites IQR
Q1 = df['Fare'].quantile(0.25)
Q3 = df['Fare'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(f"Limites de Outlier para 'Fare': Inferior = {lower_bound:.2f}, Superior = {upper_bound:.2f}")
# Remover outliers
df_cleaned = df[(df['Fare'] >= lower_bound) & (df['Fare'] <= upper_bound)]
print(f"Linhas removidas: {len(df) - len(df_cleaned)}")
# Visualização opcional depois da remoção
plt.figure(figsize=(8, 6))
df_cleaned.boxplot(column='Fare')
plt.title('Boxplot de Fare (Depois da Remoção de Outliers)')
plt.ylabel('Fare')
plt.savefig('fare_boxplot_after.png')
plt.close()
df = df_cleaned

Limites de Outlier para 'Fare': Inferior = -26.72, Superior = 65.63
Linhas removidas: 116


In [7]:
# 4. Formatação e Engenharia de Features
print("\n4. Formatação e Engenharia de Features")
# Converter 'Sex'
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
print("Coluna 'Sex' convertida para valores numéricos.")
# One-Hot Encoding em 'Embarked'
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
print("One-Hot Encoding aplicado em 'Embarked'.")
# Remover colunas irrelevantes
df.drop(['Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)
print("Colunas 'Name', 'Ticket' e 'PassengerId' removidas.")


4. Formatação e Engenharia de Features
Coluna 'Sex' convertida para valores numéricos.
One-Hot Encoding aplicado em 'Embarked'.
Colunas 'Name', 'Ticket' e 'PassengerId' removidas.


In [8]:
# 5. Escalonamento de Dados Numéricos
print("\n5. Escalonamento de Dados Numéricos")
numeric_cols = ['Age', 'SibSp', 'Parch', 'Fare']
numeric_df = df[numeric_cols].copy()
# Min-Max Scaler
min_max_scaler = MinMaxScaler()
min_max_scaled = min_max_scaler.fit_transform(numeric_df)
min_max_scaled_df = pd.DataFrame(min_max_scaled, columns=numeric_cols, index=numeric_df.index)
print("\nMin-Max Scaler (primeiras 5 linhas):")
print(min_max_scaled_df.head())
# Standard Scaler
standard_scaler = StandardScaler()
standard_scaled = standard_scaler.fit_transform(numeric_df)
standard_scaled_df = pd.DataFrame(standard_scaled, columns=numeric_cols, index=numeric_df.index)
print("\nStandard Scaler (primeiras 5 linhas):")
print(standard_scaled_df.head())
# Robust Scaler
robust_scaler = RobustScaler()
robust_scaled = robust_scaler.fit_transform(numeric_df)
robust_scaled_df = pd.DataFrame(robust_scaled, columns=numeric_cols, index=numeric_df.index)
print("\nRobust Scaler (primeiras 5 linhas):")
print(robust_scaled_df.head())
# Nota: As colunas não numéricas já foram tratadas no DataFrame 'df' original.


5. Escalonamento de Dados Numéricos

Min-Max Scaler (primeiras 5 linhas):
        Age  SibSp  Parch      Fare
0  0.271174    0.2    0.0  0.111538
2  0.321438    0.0    0.0  0.121923
3  0.434531    0.2    0.0  0.816923
4  0.434531    0.0    0.0  0.123846
5  0.310639    0.0    0.0  0.130128

Standard Scaler (primeiras 5 linhas):
        Age     SibSp     Parch      Fare
0 -0.508774  0.625606 -0.433718 -0.779117
2 -0.199954 -0.486423 -0.433718 -0.729373
3  0.494891  0.625606 -0.433718  2.599828
4  0.494891 -0.486423 -0.433718 -0.720161
5 -0.266302 -0.486423 -0.433718 -0.690071

Robust Scaler (primeiras 5 linhas):
        Age  SibSp  Parch      Fare
0 -0.241586    1.0    0.0 -0.317606
2  0.066106    0.0    0.0 -0.280322
3  0.758414    1.0    0.0  2.214956
4  0.758414    0.0    0.0 -0.273417
5  0.000000    0.0    0.0 -0.250864


In [9]:
# 6. Análise de Correlação
print("\n6. Análise de Correlação")
# Calcule a matriz de correlação no DataFrame final
final_df = df.copy()
final_df['Fare'] = robust_scaled_df['Fare'] # Usando Fare escalonado para a análise final
final_df['Age'] = robust_scaled_df['Age']
final_df['SibSp'] = robust_scaled_df['SibSp']
final_df['Parch'] = robust_scaled_df['Parch']

correlation_matrix = final_df.corr()
print("Matriz de Correlação:")
print(correlation_matrix)


6. Análise de Correlação
Matriz de Correlação:
            Survived    Pclass       Sex       Age     SibSp     Parch  \
Survived    1.000000 -0.238532  0.499888 -0.109244 -0.003156  0.098575   
Pclass     -0.238532  1.000000 -0.011810 -0.417393  0.113943  0.083451   
Sex         0.499888 -0.011810  1.000000 -0.134055  0.146035  0.289637   
Age        -0.109244 -0.417393 -0.134055  1.000000 -0.334185 -0.200915   
SibSp      -0.003156  0.113943  0.146035 -0.334185  1.000000  0.410182   
Parch       0.098575  0.083451  0.289637 -0.200915  0.410182  1.000000   
Fare        0.234422 -0.589776  0.134106  0.124214  0.370388  0.336844   
Embarked_Q  0.032707  0.214080  0.107002 -0.066755 -0.013621 -0.069729   
Embarked_S -0.102337 -0.054342 -0.095835  0.064770  0.055095  0.038364   

                Fare  Embarked_Q  Embarked_S  
Survived    0.234422    0.032707   -0.102337  
Pclass     -0.589776    0.214080   -0.054342  
Sex         0.134106    0.107002   -0.095835  
Age         0.124214   